In [4]:
import pandas as pd

## Import Data

In [ ]:
# Results from quarterly earnings reports.
financials = pd.read_csv("files/train_files/financials.csv")

# Data on the status of a variety of options based on the broader market. 
# Many options include implicit predictions of the future price of the stock 
# market and so may be of interest even though the options are not scored directly.
options = pd.read_csv("files/train_files/options.csv")

# The core dataset contains on the 2,000 most commonly traded equities but many less 
# liquid securities are also traded on the Tokyo market. This file contains data 
# for those securities, which aren't scored but may be of interest for assessing the market as a whole.
secondary_stock_prices = pd.read_csv("files/train_files/secondary_stock_prices.csv")

# The core file of interest. Includes the daily closing price for each stock and the target column.
stock_prices = pd.read_csv("files/train_files/stock_prices.csv")

# Aggregated summary of trading volumes from the previous business week.
trades = pd.read_csv("files/train_files/trades.csv")

# Mapping between the SecuritiesCode and company names, plus general information about which industry the company is in.
stock_list = pd.read_csv("files/stock_list.csv")

## Explore Data For Cleaning

In [41]:
def nullByCol(df):
    length = len(df)
    print("Total Length", length)
    for col in df.columns:
        null_count = sum(df[col].isna())
        print(col, null_count, str(round(null_count / length, 5) * 100) + "%" )

In [42]:
nullByCol(stock_prices)

Total Length 2332531
RowId 0 0.0%
Date 0 0.0%
SecuritiesCode 0 0.0%
Open 7608 0.326%
High 7608 0.326%
Low 7608 0.326%
Close 7608 0.326%
Volume 0 0.0%
AdjustmentFactor 0 0.0%
ExpectedDividend 2313666 99.191%
SupervisionFlag 0 0.0%
Target 238 0.01%


## Remove Null Values

In [ ]:
stock_prices_clean = stock_prices[stock_prices['Open'].notna()]

In [79]:
nullByCol(stock_prices_clean)

Total Length 2324923
RowId 0 0.0%
Date 0 0.0%
SecuritiesCode 0 0.0%
Open 0 0.0%
High 0 0.0%
Low 0 0.0%
Close 0 0.0%
Volume 0 0.0%
AdjustmentFactor 0 0.0%
ExpectedDividend 2306082 99.19%
SupervisionFlag 0 0.0%
Target 0 0.0%


In [80]:
print("Removed", len(stock_prices) - len(stock_prices_clean), "rows with null values in Open, Close, Low, High columns.")

Removed 7608 rows with null values in Open, Close, Low, High columns.


## Add Names and Sectors

In [104]:
stock_prices_clean = pd.merge(stock_prices_clean, stock_list[['SecuritiesCode', 'Name', 'Section/Products']], on="SecuritiesCode", how="left")
stock_prices_clean

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Name,Section/Products
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,"KYOKUYO CO.,LTD.",First Section (Domestic)
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324,"Nippon Suisan Kaisha,Ltd.",First Section (Domestic)
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154,Maruha Nichiro Corporation,First Section (Domestic)
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053,"KANEKO SEEDS CO.,LTD.",First Section (Domestic)
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026,SAKATA SEED CORPORATION,First Section (Domestic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324918,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816,SAC'S BAR HOLDINGS INC.,First Section (Domestic)
2324919,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478,GECOSS CORPORATION,First Section (Domestic)
2324920,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302,"YAMAZAWA CO.,LTD.",First Section (Domestic)
2324921,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098,YAMAYA CORPORATION,First Section (Domestic)


In [110]:
stock_prices_clean = stock_prices_clean.drop(["RowId"], axis=1)

In [111]:
nullByCol(stock_prices_clean)

Total Length 2324923
Date 0 0.0%
SecuritiesCode 0 0.0%
Open 0 0.0%
High 0 0.0%
Low 0 0.0%
Close 0 0.0%
Volume 0 0.0%
AdjustmentFactor 0 0.0%
ExpectedDividend 2306082 99.19%
SupervisionFlag 0 0.0%
Target 0 0.0%
Name 0 0.0%
Section/Products 0 0.0%


## Calculate Daily Returns and Order

In [ ]:
stock_prices_clean["Adj_Close"] = stock_prices_clean["Close"] * stock_prices_clean["AdjustmentFactor"]

In [164]:
df_adj_close = stock_prices_clean.sort_values(by=["SecuritiesCode", "Date"])
df_adj_close['Daily_Return'] = df_adj_close.groupby('SecuritiesCode')['Adj_Close'].pct_change().shift(-1)
df_adj_close = df_adj_close.sort_values(by=["Date", "Daily_Return"], ascending=[True, False])

In [170]:
df_adj_close[df_adj_close['Date'] == "2017-01-04"]

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Name,Section/Products,Adj_Close,Daily_Return
1732,2017-01-04,9474,2149.0,2156.0,2072.0,2119.0,325800,1.0,NaN,False,-0.069874,"ZENRIN CO.,LTD.",First Section (Domestic),2119.0,0.235960
114,2017-01-04,2130,505.0,518.0,500.0,510.0,130500,1.0,NaN,False,0.066556,"Members Co.,Ltd.",First Section (Domestic),510.0,0.178431
1469,2017-01-04,8157,571.0,580.0,571.0,575.0,4600,1.0,NaN,False,0.000000,"TSUZUKI DENKI CO.,LTD.",First Section (Domestic),575.0,0.120000
1142,2017-01-04,6904,495.0,499.0,485.0,494.0,78600,1.0,NaN,False,0.061818,"HARADA INDUSTRY CO.,LTD.",First Section (Domestic),494.0,0.113360
1092,2017-01-04,6753,282.0,298.0,281.0,293.0,91198000,1.0,NaN,False,-0.027692,Sharp Corporation,First Section (Domestic),293.0,0.109215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2017-01-04,3445,4160.0,4160.0,4030.0,4130.0,18300,1.0,NaN,False,0.005051,"RS Technologies Co.,Ltd.",First Section (Domestic),4130.0,-0.041162
558,2017-01-04,4246,1498.0,1550.0,1497.0,1546.0,203200,1.0,NaN,False,-0.022267,DaikyoNishikawa Corporation,First Section (Domestic),1546.0,-0.041397
1173,2017-01-04,6976,1405.0,1468.0,1401.0,1464.0,2693000,1.0,NaN,False,-0.027897,"TAIYO YUDEN CO.,LTD.",First Section (Domestic),1464.0,-0.045082
393,2017-01-04,3436,1550.0,1594.0,1543.0,1587.0,6201000,1.0,NaN,False,-0.028000,SUMCO CORPORATION,First Section (Domestic),1587.0,-0.054820


In [171]:
df_adj_close

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Name,Section/Products,Adj_Close,Daily_Return
1732,2017-01-04,9474,2149.0,2156.0,2072.0,2119.0,325800,1.0,NaN,False,-0.069874,"ZENRIN CO.,LTD.",First Section (Domestic),2119.0,0.235960
114,2017-01-04,2130,505.0,518.0,500.0,510.0,130500,1.0,NaN,False,0.066556,"Members Co.,Ltd.",First Section (Domestic),510.0,0.178431
1469,2017-01-04,8157,571.0,580.0,571.0,575.0,4600,1.0,NaN,False,0.000000,"TSUZUKI DENKI CO.,LTD.",First Section (Domestic),575.0,0.120000
1142,2017-01-04,6904,495.0,499.0,485.0,494.0,78600,1.0,NaN,False,0.061818,"HARADA INDUSTRY CO.,LTD.",First Section (Domestic),494.0,0.113360
1092,2017-01-04,6753,282.0,298.0,281.0,293.0,91198000,1.0,NaN,False,-0.027692,Sharp Corporation,First Section (Domestic),293.0,0.109215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324918,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816,SAC'S BAR HOLDINGS INC.,First Section (Domestic),528.0,NaN
2324919,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478,GECOSS CORPORATION,First Section (Domestic),794.0,NaN
2324920,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302,"YAMAZAWA CO.,LTD.",First Section (Domestic),1645.0,NaN
2324921,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098,YAMAYA CORPORATION,First Section (Domestic),2389.0,NaN
